1. Descripción de la arquitectura apoyándose en el artículo original.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.applications import MobileNet
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D

#importamos las librerías necesarias
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import keras
import tensorflow as tf
from keras.applications.imagenet_utils  import preprocess_input, decode_predictions
import numpy as np

from keras.applications.inception_v3 import InceptionV3
from keras.layers import Input

In [15]:
#Se leen y almacenan las imagenes, tambien se extrae el texto antes del primer guion bajo que contiene el nombre de la clase a la 
#que pertenece cada imagen

folder_path = "./Imagenes_para_exposiciones_Grupo_2" # Ruta carpeta de imagenes
labels=[] #Lista vacia para las etiquetas de las clases
imagenes_list = [] #Lista vacia para almacenar las imagenes 
clases_labels_num=list(range(50))

for image_name in os.listdir(folder_path):
    img = image.load_img("Imagenes_para_exposiciones_Grupo_2/"+image_name, target_size=(224,224,3))
    label=image_name.split('_')
    labels.append(label[0])
    imagenes_list.append(np.array(img))

In [20]:
def palabras_a_numeros(lista_palabras):
    # Creamos un diccionario para mapear palabras a números
    palabra_numero = {}
    numeros = []
    contador = 1  # Empezamos desde 1

    for palabra in lista_palabras:
        # Si la palabra no está en el diccionario, le asignamos un número único
        if palabra not in palabra_numero:
            palabra_numero[palabra] = contador
            contador += 1
        # Agregamos el número correspondiente a la palabra a la lista
        numeros.append(palabra_numero[palabra])

    return numeros

In [21]:
labels=palabras_a_numeros(labels)
clases=(list(set(labels)))

In [40]:
len(clases)

44

In [31]:
labels

array([ 1,  1,  2,  2,  2,  3,  3,  3,  4,  4,  4,  5,  5,  5,  6,  6,  7,
        7,  8,  8,  8,  8,  9,  9,  9,  9, 10, 11, 11, 11, 12, 12, 13, 13,
       13, 14, 14, 15, 15, 16, 17, 18, 18, 19, 19, 19, 19, 19, 20, 20, 21,
       21, 22, 23, 23, 24, 24, 24, 24, 25, 25, 25, 26, 26, 27, 27, 28, 29,
       30, 30, 30, 31, 31, 31, 31, 32, 33, 34, 35, 35, 36, 37, 37, 37, 37,
       37, 38, 38, 39, 40, 41, 42, 42, 42, 42, 43, 44, 44, 44, 44])

In [48]:
np.random.seed(57)
seeed=57

images = np.array(imagenes_list)
labels = np.array(labels)

images = images.astype(np.uint8)
labels = labels.astype(np.int32)

In [54]:
images.shape

(100, 224, 224, 3)

In [58]:
len(labels)

100

In [64]:
labels=labels.reshape(-1)

In [65]:
labels

array([ 1,  1,  2,  2,  2,  3,  3,  3,  4,  4,  4,  5,  5,  5,  6,  6,  7,
        7,  8,  8,  8,  8,  9,  9,  9,  9, 10, 11, 11, 11, 12, 12, 13, 13,
       13, 14, 14, 15, 15, 16, 17, 18, 18, 19, 19, 19, 19, 19, 20, 20, 21,
       21, 22, 23, 23, 24, 24, 24, 24, 25, 25, 25, 26, 26, 27, 27, 28, 29,
       30, 30, 30, 31, 31, 31, 31, 32, 33, 34, 35, 35, 36, 37, 37, 37, 37,
       37, 38, 38, 39, 40, 41, 42, 42, 42, 42, 43, 44, 44, 44, 44])

In [66]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size = 0.2, random_state = seeed)
print("x_train shape = "  ,x_train.shape)
print("y_train shape = "  ,y_train.shape)
print("\nx_test shape = " ,x_test.shape)
print("y_test shape = "   ,y_test.shape)

x_train shape =  (80, 224, 224, 3)
y_train shape =  (80,)

x_test shape =  (20, 224, 224, 3)
y_test shape =  (20,)


2. Implementación (incluyendo argumentos para iniciar el modelo con Keras)

In [41]:
modelo_mobilenet = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3)) #Modelo sin las capas densas (top)

#capas densas (totalmente conectadas) en la parte superior

#Capa de agrupación global y capa densa (totalmente conectada) en la parte superior del modelo
x = modelo_mobilenet.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x) 
predictions = Dense(len(clases), activation='softmax')(x)  

modelo_final = Model(inputs=modelo_mobilenet.input, outputs=predictions)

modelo_final.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

modelo_final.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                    

In [69]:
preds=modelo_final.predict(x_train)

3/3 [==============================] - 0s 136ms/step


In [75]:
len(preds)
#preds

80

3. Aplicación del modelo en el conjunto de imágenes asignadas al grupo.
Discutir los resultados mostrando las clases presentes en sus datos.